In [1]:
import os,warnings,gc
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import polars as pl
# import polars.selectors as cs
# from glob import glob
from sklearn.metrics import roc_auc_score,auc
from sklearn.model_selection import StratifiedGroupKFold,train_test_split
# import lightgbm as lgb
import xgboost as xgb
import catboost as cgb

In [2]:
pl.Config.set_float_precision(2)
pl.Config.set_fmt_float("full")
pl.Config.set_tbl_rows(10)
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_streaming_chunk_size(10)

polars.config.Config

In [12]:
total_imp_df = (
    pl.read_parquet("/home/sohail/Downloads/lgb_imp.parquet")
    .select(pl.col("index"),pl.mean_horizontal(pl.col(pl.NUMERIC_DTYPES)).alias("mean_lgb"))
).join(
    pl.read_parquet("/home/sohail/Downloads/cat_imp.parquet")
    .select(pl.col("index"),pl.mean_horizontal(pl.col(pl.NUMERIC_DTYPES)).alias("mean_cat")),
    on="index"
)
total_imp_df.head()

index,mean_lgb,mean_cat
str,f64,f64
"""weeknum""",1024.01,0.25
"""month""",1086.04,0.03
"""weekday""",10226.59,0.30
"""week""",1938.57,0.09
"""year""",0.00,0.00


In [37]:
(
    total_imp_df
    .filter(
        (pl.col("mean_lgb") > 5000) & (pl.col("mean_cat") > 0.01) & (pl.col("index").str.split("_").list.lengths() > 2)
    )
    .with_columns(
        pl.when(
            pl.col("index").str.split("_").list.get(0).str.contains_any(["year","week","month"])
        )
        .then(
            pl.col("index").str.split("_").list.get(1)
        )
        .otherwise(
            pl.col("index").str.split("_").list.get(0)
        )
        .alias("prefix_string")
    )
    .with_columns(
        pl.when(
            pl.col("index").str.split("_").list.get(0).str.contains_any(["year","week","month"])
        )
        .then(
            pl.col("index").str.split("_").list.get(2)
        )
        .otherwise(
            pl.col("index").str.split("_").list.get(1)
        )
        .alias("type")
    )
    .with_columns(
        pl.when(
            pl.col("index").str.split("_").list.get(0).str.contains_any(["week","year","month"])
        )
        .then(
            pl.col("index").str.split("_").list.slice(3).list.join("_")
        )
        .otherwise(
            pl.col("index").str.split("_").list.slice(2).list.join("_")
        )
        .alias("col_name")
    )
)

index,mean_lgb,mean_cat,prefix_string,type,col_name
str,f64,f64,str,str,str
"""pastshallow_mean_byoccupationinc_3656910L""",10532.33,0.10,"""pastshallow""","""mean""","""byoccupationinc_3656910L"""
"""pastshallow_mean_childnum_21L""",14169.74,0.16,"""pastshallow""","""mean""","""childnum_21L"""
"""pastshallow_mean_pmtnum_8L""",16390.84,0.53,"""pastshallow""","""mean""","""pmtnum_8L"""
"""pastshallow_mean_tenor_203L""",17005.42,0.12,"""pastshallow""","""mean""","""tenor_203L"""
"""pastshallow_mean_annuity_853A""",9079.77,0.29,"""pastshallow""","""mean""","""annuity_853A"""
…,…,…,…,…,…
"""week_intshallow_min_numberofoverdueinstlmaxdat_148D""",7091.89,0.11,"""intshallow""","""min""","""numberofoverdueinstlmaxdat_148D"""
"""week_intshallow_min_numberofoverdueinstlmaxdat_641D""",7741.78,0.06,"""intshallow""","""min""","""numberofoverdueinstlmaxdat_641D"""
"""week_intshallow_min_overdueamountmax2date_1002D""",7545.68,0.12,"""intshallow""","""min""","""overdueamountmax2date_1002D"""


In [3]:
total_df = pl.read_parquet("/home/sohail/total_df.parquet").select(pl.all().shrink_dtype())
total_df.head()

weeknum,month,weekday,week,year,target,pastshallow_mean_num_group1,pastshallow_mean_byoccupationinc_3656910L,pastshallow_mean_childnum_21L,pastshallow_mean_credacc_transactions_402L,pastshallow_mean_pmtnum_8L,pastshallow_mean_tenor_203L,pastshallow_mean_actualdpd_943P,pastshallow_mean_annuity_853A,pastshallow_mean_credacc_actualbalance_314A,pastshallow_mean_credacc_credlmt_575A,pastshallow_mean_credacc_maxhisbal_375A,pastshallow_mean_credacc_minhisbal_90A,pastshallow_mean_credamount_590A,pastshallow_mean_currdebt_94A,pastshallow_mean_downpmt_134A,pastshallow_mean_mainoccupationinc_437A,pastshallow_mean_maxdpdtolerance_577P,pastshallow_mean_outstandingdebt_522A,pastshallow_mean_revolvingaccount_394A,pastshallow_max_num_group1,pastshallow_max_byoccupationinc_3656910L,pastshallow_max_childnum_21L,pastshallow_max_credacc_transactions_402L,pastshallow_max_pmtnum_8L,pastshallow_max_tenor_203L,pastshallow_max_actualdpd_943P,pastshallow_max_annuity_853A,pastshallow_max_credacc_actualbalance_314A,pastshallow_max_credacc_credlmt_575A,pastshallow_max_credacc_maxhisbal_375A,pastshallow_max_credacc_minhisbal_90A,…,week_intdepth_min_pmts_month_158D,week_intdepth_min_pmts_month_706D,week_intdepth_min_pmts_year_1139D,week_intdepth_min_pmts_year_507D,week_extshallow_mean_contractdate_551D,week_extshallow_mean_contractmaturitydate_151D,week_extshallow_mean_dpdmaxdatemonth_804D,week_extshallow_mean_dpdmaxdateyear_742D,week_extshallow_mean_lastupdate_260D,week_extshallow_mean_overdueamountmaxdatemonth_494D,week_extshallow_mean_overdueamountmaxdateyear_432D,week_extshallow_max_contractdate_551D,week_extshallow_max_contractmaturitydate_151D,week_extshallow_max_dpdmaxdatemonth_804D,week_extshallow_max_dpdmaxdateyear_742D,week_extshallow_max_lastupdate_260D,week_extshallow_max_overdueamountmaxdatemonth_494D,week_extshallow_max_overdueamountmaxdateyear_432D,week_extshallow_min_contractdate_551D,week_extshallow_min_contractmaturitydate_151D,week_extshallow_min_dpdmaxdatemonth_804D,week_extshallow_min_dpdmaxdateyear_742D,week_extshallow_min_lastupdate_260D,week_extshallow_min_overdueamountmaxdatemonth_494D,week_extshallow_min_overdueamountmaxdateyear_432D,week_extdepth_mean_pmts_date_1107D,week_extdepth_max_pmts_date_1107D,week_extdepth_min_pmts_date_1107D,week_rega_mean_recorddate_4527225D,week_rega_max_recorddate_4527225D,week_rega_min_recorddate_4527225D,week_regb_mean_deductiondate_4917603D,week_regb_max_deductiondate_4917603D,week_regb_min_deductiondate_4917603D,week_regc_mean_deductiondate_4917603D,week_regc_max_deductiondate_4917603D,week_regc_min_deductiondate_4917603D
u8,i8,i8,i8,i8,u8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8
0,1,4,1,1,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
0,1,4,1,1,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
0,1,5,1,1,0,0.50,null,0.00,null,18.00,18.00,0.00,1161.30,null,0.00,null,null,13000.00,null,0.00,8200.00,null,null,null,1,null,0.00,null,24.00,24.00,0.00,1682.40,null,0.00,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
0,1,4,1,1,0,0.00,null,null,null,12.00,12.00,0.00,6140.00,null,0.00,null,n

In [4]:
# week_num = total_df["weeknum"]
# train_total = total_df.drop("weeknum")
# gc.collect()

# def filter_ind(df:pl.DataFrame,indexes:np.array):
#     return (
#         df
#         .with_row_index()
#         .filter(
#             pl.col("index").is_in(indexes)
#         )
#         .drop("index")
#     )

In [5]:
# class_params = {
#     "objective":"binary",
#     "boosting_type": "gbdt",
#     "metric":"auc",
#     "max_depth": 128,
#     "learning_rate": 0.01,
#     "n_estimators": 10000,
#     "colsample_bynode": 0.8,
#     "colsample_bytree": 0.8,
#     "random_state": 420,
#     "reg_alpha": 0.15,
#     "reg_lambda": 15,
#     "extra_trees": True,
#     "num_leaves": 256,
#     "device": "gpu",
#     "importance_type": "gain",
#     "verbose": -1
# }

# xgb_params = {
#     "objective": "binary:logistic",
#     "n_estimators": 10000,
#     "eval_metric": "auc",
#     "seed": 420,
#     "booster": "gbtree",
#     "device": "cuda",
#     "eta": 0.01,
#     "gamma": 5,
#     "max_depth": 128,
#     "colsample_bytree": 0.8,
#     "colsample_bynode": 0.8,
#     "lambda": 10,
#     "alpha": 2,
#     "updater": "grow_gpu_hist",
#     "grow_policy": "depthwise",
#     "max_leaves": 256,
#     "num_parallel_tree": 1,    
#     "sample_type": "uniform",
#     "normalize_type": "tree",
#     "rate_drop": 0.15,
#     "skip_drop": 0.9,
#     "enable_categorical": True
# }

# cat_params = {
#     "eval_metric": "AUC",
#     "task_type": "GPU",
#     "iterations": 10000,
#     "learning_rate": 0.01,
#     "bootstrap_type": "Poisson",
#     "random_seed": 420,
#     "l2_leaf_reg": 15,
#     "subsample": 0.8,
#     "depth": 32,
#     "max_leaves": 64,
#     "grow_policy": "Lossguide",
#     "od_type": "Iter",
#     "od_wait": 1000,
#     "verbose": 500
# }

In [6]:
# cv = StratifiedGroupKFold(n_splits=5,shuffle=False)
# lgb_imp = pd.DataFrame(index=total_df.drop("target").columns,columns=["gain_1","gain_2","gain_3","gain_4","gain_5"])

# for i,(train_ind,valid_ind) in enumerate(
#     cv.split(total_df,total_df.select("target"),
#     groups=week_num)):
#     print(f"Training start for LGBClassifier: {i+1}")
#     lgb_model = lgb.LGBMClassifier(**class_params)
#     lgb_model.fit(
#         (
#             total_df
#             .drop("target")
#             .pipe(filter_ind,train_ind)
#         ),
#         (
#             total_df
#             .select('target')
#             .pipe(filter_ind,train_ind)
#         ),
#         eval_set=[(
#             (
#                 total_df
#                 .drop("target")
#                 .pipe(filter_ind,valid_ind)
#             ),
#             (
#                 total_df
#                 .select("target")
#                 .pipe(filter_ind,valid_ind)
#             )
#         )],
#         callbacks=[lgb.log_evaluation(500),lgb.early_stopping(800)]
#     )
#     for imp,col_name in sorted(zip(lgb_model.feature_importances_,total_df.drop("target").columns)):
#         lgb_imp.loc[col_name,f"gain_{i+1}"] = imp
#     gc.collect()

# lgb_imp.reset_index(inplace=True)
# lgb_imp.to_parquet("/home/sohail/Downloads/lgb_imp.parquet")

# del train_ind,valid_ind,lgb_model
# gc.collect()

# xgb_imp = pd.DataFrame(index=total_df.drop("target").columns,columns=["gain_1","gain_2","gain_3","gain_4","gain_5"])

# for i,(train_ind,valid_ind) in enumerate(
#     cv.split(total_df,total_df.select("target"),
#     groups=week_num)):
#     print(f"Training start for XGBClassifier: {i+1}")
#     early_stop = xgb.callback.EarlyStopping(rounds=800)
#     log_eval = xgb.callback.EvaluationMonitor(period=500)
#     xgb_model = xgb.XGBClassifier(**xgb_params)
#     xgb_model.fit(
#         (
#             total_df
#             .drop("target")
#             .pipe(filter_ind,train_ind)
#         ),
#         (
#             total_df
#             .select("target")
#             .pipe(filter_ind,train_ind)
#         ),
#         eval_set=[(
#             (
#                 total_df
#                 .drop("target")
#                 .pipe(filter_ind,valid_ind)
#             ),
#             (
#                 total_df
#                 .select("target")
#                 .pipe(filter_ind,valid_ind)
#             )
#         )],
#         callbacks=[early_stop,log_eval],
#         verbose=False
#     )
#     for imp,col_name in sorted(zip(xgb_model.feature_importances_,total_df.drop("target").columns)):
#         xgb_imp.loc[col_name,f"gain_{i+1}"] = imp
#     gc.collect()

# xgb_imp.reset_index(inplace=True)
# xgb_imp.to_parquet("/home/sohail/Downloads/xgb_imp.parquet")

# del xgb_model,train_ind,valid_ind,log_eval,early_stop
# gc.collect()

# cat_imp = pd.DataFrame(index=total_df.drop("target").columns,columns=["gain_1","gain_2","gain_3","gain_4","gain_5"])

# for i,(train_ind,valid_ind) in enumerate(
#     cv.split(total_df,total_df.select("target"),
#     groups=week_num)):
#     print(f"Training start for CatBoostClassifier: {i+1}")
#     cat_model = cgb.CatBoostClassifier(**cat_params)
#     cat_model.fit(
#         (
#             total_df
#             .drop("target")
#             .pipe(filter_ind,train_ind)
#             .to_pandas()
#         ),
#         (
#             total_df
#             .select("target")
#             .pipe(filter_ind,train_ind)
#             .to_pandas()
#         ),
#         eval_set=[(
#             (
#                 total_df
#                 .drop("target")
#                 .pipe(filter_ind,valid_ind)
#                 .to_pandas()
#             ),
#             (
#                 total_df
#                 .select("target")
#                 .pipe(filter_ind,valid_ind)
#                 .to_pandas()
#             )
#         )]
#     )
#     for imp,col_name in sorted(zip(cat_model.feature_importances_,total_df.drop("target").columns)):
#         cat_imp.loc[col_name,f"gain_{i+1}"] = imp
#     gc.collect()

# cat_imp.reset_index(inplace=True)
# cat_imp.to_parquet("/home/sohail/Downloads/cat_imp.parquet")

# del total_df,cat_model,train_ind,valid_ind

Training start for CatBoostClassifier: 1


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7323337	best: 0.7323337 (0)	total: 146ms	remaining: 24m 24s
500:	test: 0.8430641	best: 0.8430641 (500)	total: 59.1s	remaining: 18m 41s
1000:	test: 0.8514374	best: 0.8514374 (1000)	total: 1m 57s	remaining: 17m 34s
1500:	test: 0.8544928	best: 0.8544928 (1500)	total: 2m 50s	remaining: 16m 5s
2000:	test: 0.8560014	best: 0.8560014 (2000)	total: 3m 38s	remaining: 14m 35s
2500:	test: 0.8568877	best: 0.8568877 (2500)	total: 4m 24s	remaining: 13m 11s
3000:	test: 0.8575479	best: 0.8575479 (3000)	total: 5m 7s	remaining: 11m 56s
3500:	test: 0.8579703	best: 0.8579771 (3492)	total: 5m 48s	remaining: 10m 46s
4000:	test: 0.8583377	best: 0.8583377 (4000)	total: 6m 29s	remaining: 9m 43s
4500:	test: 0.8585808	best: 0.8585808 (4500)	total: 7m 9s	remaining: 8m 45s
5000:	test: 0.8587766	best: 0.8587778 (4997)	total: 7m 49s	remaining: 7m 49s
5500:	test: 0.8590264	best: 0.8590275 (5496)	total: 8m 30s	remaining: 6m 57s
6000:	test: 0.8592649	best: 0.8592666 (5998)	total: 9m 10s	remaining: 6m 6s
6500:

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7320620	best: 0.7320620 (0)	total: 102ms	remaining: 17m 4s
500:	test: 0.8444090	best: 0.8444090 (500)	total: 59.8s	remaining: 18m 53s
1000:	test: 0.8523053	best: 0.8523053 (1000)	total: 1m 58s	remaining: 17m 48s
1500:	test: 0.8552133	best: 0.8552133 (1500)	total: 2m 52s	remaining: 16m 19s
2000:	test: 0.8567263	best: 0.8567263 (2000)	total: 3m 42s	remaining: 14m 50s
2500:	test: 0.8576097	best: 0.8576097 (2500)	total: 4m 28s	remaining: 13m 24s
3000:	test: 0.8582340	best: 0.8582340 (3000)	total: 5m 12s	remaining: 12m 7s
3500:	test: 0.8586814	best: 0.8586816 (3498)	total: 5m 54s	remaining: 10m 57s
4000:	test: 0.8590473	best: 0.8590483 (3998)	total: 6m 35s	remaining: 9m 53s
4500:	test: 0.8593991	best: 0.8593992 (4484)	total: 7m 16s	remaining: 8m 52s
5000:	test: 0.8596373	best: 0.8596380 (4994)	total: 7m 56s	remaining: 7m 56s
5500:	test: 0.8598614	best: 0.8598643 (5486)	total: 8m 36s	remaining: 7m 2s
6000:	test: 0.8600376	best: 0.8600411 (5992)	total: 9m 16s	remaining: 6m 10s
6500

...

0:	test: 0.7379596	best: 0.7379596 (0)	total: 96ms	remaining: 16m
500:	test: 0.8493344	best: 0.8493344 (500)	total: 59.6s	remaining: 18m 49s
1000:	test: 0.8576453	best: 0.8576453 (1000)	total: 1m 59s	remaining: 17m 51s
1500:	test: 0.8605820	best: 0.8605820 (1500)	total: 2m 53s	remaining: 16m 24s
2000:	test: 0.8621259	best: 0.8621283 (1998)	total: 3m 43s	remaining: 14m 53s
2500:	test: 0.8629549	best: 0.8629549 (2500)	total: 4m 29s	remaining: 13m 28s
3000:	test: 0.8635453	best: 0.8635453 (3000)	total: 5m 13s	remaining: 12m 10s
3500:	test: 0.8639877	best: 0.8639898 (3499)	total: 5m 55s	remaining: 10m 59s
4000:	test: 0.8643256	best: 0.8643256 (4000)	total: 6m 36s	remaining: 9m 54s
4500:	test: 0.8645987	best: 0.8646007 (4489)	total: 7m 18s	remaining: 8m 55s
5000:	test: 0.8648611	best: 0.8648629 (4999)	total: 7m 58s	remaining: 7m 58s
5500:	test: 0.8651255	best: 0.8651257 (5490)	total: 8m 39s	remaining: 7m 4s
6000:	test: 0.8654025	best: 0.8654054 (5997)	total: 9m 19s	remaining: 6m 12s
6500:	t

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7317250	best: 0.7317250 (0)	total: 92.5ms	remaining: 15m 24s
500:	test: 0.8481480	best: 0.8481480 (500)	total: 59.5s	remaining: 18m 47s
1000:	test: 0.8564734	best: 0.8564734 (1000)	total: 1m 58s	remaining: 17m 47s
1500:	test: 0.8594918	best: 0.8594918 (1500)	total: 2m 53s	remaining: 16m 20s
2000:	test: 0.8609800	best: 0.8609810 (1999)	total: 3m 42s	remaining: 14m 51s
2500:	test: 0.8619441	best: 0.8619441 (2500)	total: 4m 29s	remaining: 13m 27s
3000:	test: 0.8626906	best: 0.8626906 (3000)	total: 5m 13s	remaining: 12m 11s
3500:	test: 0.8632199	best: 0.8632215 (3497)	total: 5m 56s	remaining: 11m 1s
4000:	test: 0.8636064	best: 0.8636065 (3999)	total: 6m 38s	remaining: 9m 57s
4500:	test: 0.8638951	best: 0.8638951 (4500)	total: 7m 19s	remaining: 8m 57s
5000:	test: 0.8641142	best: 0.8641142 (5000)	total: 8m	remaining: 8m
5500:	test: 0.8643894	best: 0.8643898 (5498)	total: 8m 41s	remaining: 7m 6s
6000:	test: 0.8645885	best: 0.8645916 (5996)	total: 9m 22s	remaining: 6m 14s
6500:	test

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7343213	best: 0.7343213 (0)	total: 92.5ms	remaining: 15m 25s
500:	test: 0.8440589	best: 0.8440589 (500)	total: 59.5s	remaining: 18m 48s
1000:	test: 0.8527435	best: 0.8527435 (1000)	total: 1m 58s	remaining: 17m 41s
1500:	test: 0.8559710	best: 0.8559710 (1500)	total: 2m 51s	remaining: 16m 12s
2000:	test: 0.8575150	best: 0.8575150 (2000)	total: 3m 40s	remaining: 14m 42s
2500:	test: 0.8585345	best: 0.8585356 (2499)	total: 4m 26s	remaining: 13m 18s
3000:	test: 0.8592023	best: 0.8592023 (3000)	total: 5m 9s	remaining: 12m 2s
3500:	test: 0.8597208	best: 0.8597208 (3500)	total: 5m 51s	remaining: 10m 53s
4000:	test: 0.8601185	best: 0.8601229 (3994)	total: 6m 33s	remaining: 9m 49s
4500:	test: 0.8603760	best: 0.8603769 (4499)	total: 7m 14s	remaining: 8m 50s
5000:	test: 0.8606377	best: 0.8606377 (5000)	total: 7m 55s	remaining: 7m 54s
5500:	test: 0.8608270	best: 0.8608270 (5500)	total: 8m 35s	remaining: 7m 1s
6000:	test: 0.8609867	best: 0.8609875 (5999)	total: 9m 14s	remaining: 6m 9s
6500